In [1]:
import glob
import os
import numpy as np
import datetime
from netCDF4 import Dataset
import scipy.io
import h5py
from pyproj import Proj, Transformer
import cartopy.crs as ccrs
from scipy.spatial.kdtree import KDTree
import scipy.stats as scst
from regrid import regrid
from regrid import regrid_fast
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
import cartopy
import warnings
warnings.filterwarnings("ignore")

/var/folders/_1/fh2vf5m57871c546k_sf7xr80000gn/T/ipykernel_8967/571042888.py:10: DeprecationWarning: Please use `KDTree` from the `scipy.spatial` namespace, the `scipy.spatial.kdtree` namespace is deprecated.
  from scipy.spatial.kdtree import KDTree
/opt/anaconda3/envs/phd/lib/python3.8/site-packages/pyproj/crs.py:77: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method.
  return _prepare_from_string(" ".join(pjargs))


### function to find nearest EASE grid cell for OIB coords

In [2]:
def lonlat_to_xy(coords_1, coords_2, hemisphere, inverse=False):

    """Converts between longitude/latitude and EASE xy coordinates.
 
    Args:
        lon (float): WGS84 longitude
        lat (float): WGS84 latitude
        hemisphere (string): 'n' or 's'
        inverse (bool): if true, converts xy to lon/lat
 
    Returns:
        tuple: pair of xy or lon/lat values
    """

    EASE_Proj = {'n': 'EPSG:3408',
                 's': 'EPGS:3409'}
    
    WGS_Proj = 'EPSG:4326'
    
    for coords in [coords_1, coords_2]: assert isinstance(coords,(np.ndarray,list))

    if inverse == False: # lonlat to xy
        
        lon, lat = coords_1, coords_2
        
        transformer = Transformer.from_crs(WGS_Proj, EASE_Proj[hemisphere])
        
        x, y = transformer.transform(lat, lon)
        
        return (x, y)

    else: # xy to lonlat
        
        x, y = coords_1, coords_2
        
        transformer = Transformer.from_crs(EASE_Proj[hemisphere], WGS_Proj)
        
        lat, lon = transformer.transform(x, y)
        
        return (lon, lat)

### load OIB data, regrid and combine into one file

In [3]:
# load ease lons,lats:
ease_lats = np.load('/Users/carmennab/Dropbox/alpha_retracker/data/auxiliary/lat_25km_cent.npy')
ease_lons = np.load('/Users/carmennab/Dropbox/alpha_retracker/data/auxiliary/lon_25km_cent.npy')

# convert into x,y:
ease_x, ease_y = lonlat_to_xy(ease_lons,ease_lats,hemisphere='n')

# create KDTree to find EASE grid cell closest to buoy:
tree = KDTree(list(zip(ease_x.ravel(),ease_y.ravel())))

In [ ]:
days = np.arange(0,4018)
dates = [datetime.date(2010,1,1)+datetime.timedelta(days=np.int(days)) for days in days]

datapath = '/Users/carmennab/Dropbox/sabrina_oib/raw_data/'

#files = sorted(glob.glob(datapath+'*.txt'))

for day in days:
    oib_sit = np.full((ease_x.shape),np.nan)
    oib_snow = np.full((ease_x.shape),np.nan)
    oib_laser = np.full((ease_x.shape),np.nan)
    
    inds_si = {}
    inds_sn = {}
    inds_la = {}
    date = dates[day].strftime('%Y%m%d')
 
    for f in os.listdir(datapath):
        if date in f:
            try:
                data = pd.read_csv(datapath+f,delimiter=',',usecols=[0,1,2,5,7])

                sit = np.array(data['thickness'])
                snow = np.array(data['snow_depth'])
                laser = np.array(data['ATM_fb'])
                sit[sit<0] = np.nan
                snow[snow<0] = np.nan
                laser[laser<0] = np.nan

                oib_lons = np.array(data['lon'])
                oib_lats = np.array(data['lat'])
                oib_x,oib_y = lonlat_to_xy(oib_lons,oib_lats,hemisphere='n') 

                for x,y,si,sn,la in zip(oib_x,oib_y,sit,snow,laser):

                    dist, ind = tree.query([x,y])

                    ind2d = np.unravel_index(ind, (ease_lats.shape[0],ease_lats.shape[1]))

                    if ind2d in inds_si.keys():
                        inds_si[ind2d].append(si)
                        inds_sn[ind2d].append(sn)
                        inds_la[ind2d].append(la)
                    else:   
                        inds_si[ind2d] = [si]
                        inds_sn[ind2d] = [sn]
                        inds_la[ind2d] = [la]

                for key in inds_si.keys():
                    oib_sit[key[0],key[1]]=np.nanmean(inds_si[key])
                    oib_snow[key[0],key[1]]=np.nanmean(inds_sn[key])
                    oib_laser[key[0],key[1]]=np.nanmean(inds_la[key])

                if ~np.isnan(oib_sit.all()) or ~np.isnan(oib_snow.all()):
                    filepath = '/Users/carmennab/Dropbox/alpha_retracker/data/OIB_EASE/'+date+'.nc'
                    ! rm {filepath} 
                    data_vars = {'Sea Ice Thickness':(['x','y'],oib_sit),
                                 'Snow Depth':(['x','y'],oib_snow),
                                 'Laser Freeboard':(['x','y'],oib_laser)}
                    ds = xr.Dataset(data_vars = data_vars,
                                    coords={'Longitude':(['x','y'],ease_lons),
                                            'Latitude':(['x','y'],ease_lats)},
                                    )
                    ds.to_netcdf(filepath,'w')
            except:
                print('out of bounds:',date)

rm: /Users/carmennab/Dropbox/alpha_retracker/data/OIB_EASE/20120314.nc: No such file or directory
rm: /Users/carmennab/Dropbox/alpha_retracker/data/OIB_EASE/20120315.nc: No such file or directory
rm: /Users/carmennab/Dropbox/alpha_retracker/data/OIB_EASE/20120316.nc: No such file or directory
rm: /Users/carmennab/Dropbox/alpha_retracker/data/OIB_EASE/20120317.nc: No such file or directory
rm: /Users/carmennab/Dropbox/alpha_retracker/data/OIB_EASE/20120319.nc: No such file or directory
rm: /Users/carmennab/Dropbox/alpha_retracker/data/OIB_EASE/20120321.nc: No such file or directory
rm: /Users/carmennab/Dropbox/alpha_retracker/data/OIB_EASE/20120322.nc: No such file or directory
rm: /Users/carmennab/Dropbox/alpha_retracker/data/OIB_EASE/20120323.nc: No such file or directory
rm: /Users/carmennab/Dropbox/alpha_retracker/data/OIB_EASE/20120326.nc: No such file or directory
rm: /Users/carmennab/Dropbox/alpha_retracker/data/OIB_EASE/20120327.nc: No such file or directory
rm: /Users/carmennab